# Testing Camera Attacks

In this notebook we will test the efficacy of several attacks. We will begin with the line drawing attack and move on to the blurring attack. We expect to see a significant decrease in classification accuracy after using these attacks.

After this, we will attempt to boost the classification accuracy.

The classification accuracy of our initial model on the un-altered tests data was over $97\%$

In [1]:
# STDLIB
import os
import typing
import csv
import random
# Packages
import numpy as np
import cv2 as cv
import tensorflow as tf
from tensorflow import keras


In [2]:
DATA_DIR = "GTSRB"
TRAINING_DATA_PATH = os.path.join(DATA_DIR, "Final_Training/Images")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  4


## Line Drawing

This first attack is meant to simulate a crack in the camera lense of the device taking pictures of the traffic signs. 

In [3]:
def minimum_distance(p1, p2):

    i1, i2 = int(p1[0]), int(p2[0])

    # if both x-coordinates are floats and they have the same integer part
    if i1 != p1[0] and i2 != p2[0] and i1 == i2:

        # find the decimal parts
        d1, d2 = p1[0] - i1, p2[0] - i2

        # find the smaller "C"
        x = min(d1 + d2, (1-d1) + (1-d2))

        # add the y distance to the "C" distance
        return abs(p1[1] - p2[1]) + x

    # repeat with the "y-coordinates are floats" case
    i1, i2 = int(p1[1]), int(p2[1])
    if i1 != p1[1] and i2 != p2[1] and i1 == i2:
        d1, d2 = p1[1] - i1, p2[1] - i2
        y = min(d1 + d2, (1-d1) + (1-d2))
        return abs(p1[0] - p2[0]) + y

    # simple case, return the Manhattan distance
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def insert_lines(image) :
    image_data = image
    unmodified_input_image = image_data
    
    #understand the image dimensions
    image_height = image_data.shape[0]
    image_width = image_data.shape[1]
    
    #define the min legth of the line to be drawn on the image
    min_length = round(0.25 * image_height)

    line_length = 0
    iter = 0 
    while (line_length < min_length) and (iter < 10) :
        x = [random.randint(0, image_height), random.randint(0, image_width)]
        y = [random.randint(0, image_height), random.randint(0, image_width)]
        line_length = minimum_distance(x, y)
        iter = iter + 1

    random_col = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    cv.line(image_data, x, y, random_col, 1)

    output_image = cv.addWeighted(image_data, 0.15, unmodified_input_image, 0.85, 0)
    return output_image


In [8]:
def readTrafficSignsTest(rootpath: str) -> typing.Tuple[list[cv.Mat], list[str]]:
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = [] # images
    labels = [] # corresponding labels
    # loop over all 42 classes
    with open(os.path.join(rootpath, 'GT-final_test.csv')) as gtFile:
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        next(gtReader) # skip header
        # loop over all images in current annotations file
        for row in gtReader:
            file_link = os.path.join(rootpath, row[0])
            image = cv.imread(file_link) # read image, the 1th column is the filename
            images.append(image)
            labels.append(row[7]) # the 8th column is the label
    return images, labels

def run_model_on_data(model: keras.Model, images: list[cv.Mat], labels: list[str]) -> dict[str, float]:
    """_summary_: Runs the model on the given images and labels.

    Args:
        model (keras.Model): The model to run on the images.
        images (list[cv.Mat]): The images to run on. These should already be processed.
        labels (list[str]): The labels to run on.

    Returns:
        A dictionary with two keys: 'accuracy' and 'loss'.
    """
    test_images = np.array(images).astype(np.float32) # this allows us to convert it to a tensor
    test_images = test_images/255 # Everyone seems to do this so we will too!!!
    test_labels = np.array(labels).astype(np.float32)

    assert len(test_images) == len(test_labels)

    result = model.evaluate(test_images, test_labels)
    return dict(zip(model.metrics_names, result))

def process_images(images: list[cv.Mat]):
    output = []
    for image in images:
        image_with_line = insert_lines(image)
        resized_image = cv.resize(image_with_line, (32,32)) # resize to 32x32
        output.append(resized_image) # resize to 32x32
    return output

In [9]:
TEST_DATA_PATH = os.path.join(DATA_DIR, "Final_Test/Images")
MODEL_NAME = "cnn_v3.h5"

# Load the Model
model = None
if os.path.exists(MODEL_NAME):
    print("Loading existing model. If you want to retrain, delete the file: " + MODEL_NAME)
    model = keras.models.load_model(MODEL_NAME)
else:
    print("No existing model found.")
    exit(1)

real_test_images, real_test_labels = readTrafficSignsTest(TEST_DATA_PATH)
processed_test_images = process_images(real_test_images)

result_dict = run_model_on_data(model, processed_test_images, real_test_labels)
print(result_dict)

Loading existing model. If you want to retrain, delete the file: cnn_v3.h5
 25/395 [>.............................] - ETA: 1s - loss: 0.3329 - accuracy: 0.9475

/u/dej3tc/Documents/ML/GTSRB_ML_Exploration/env/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


395/395 [==============================] - 2s 5ms/step - loss: 0.3638 - accuracy: 0.9459
{'loss': 0.36384642124176025, 'accuracy': 0.9459224343299866}


### Results

Adding lines to the images after standardizing their size to 32x32 resulted in drop in accuracy to $93.87\%$

Adding lines to the images before standardizing their size to 32x32 resulted in drop in accuracy to $94.6\%$